通过该函数可以计算每日全A市场个股创新高比例，并生成创新高股票的清单，基于此函数可以作为生成创新高个股的策略基础

In [12]:
from jqdata import *
import numpy as np
import pandas as pd

In [13]:
end_date = '2022-05-23'
check_days = 15 # end_date之前的15个交易日
window = 252  # 此参数为创新高统计周期
gap = 60  # 此参数为创新高间隔，即间隔期内未突破前期新高，只在今天创下新高

In [14]:
%%time

# 起始的时间点
by_date = get_trade_days(end_date=end_date, count=window+check_days)[0]

# 确保end_date之前的window+check_days个交易日已经上市，从而确保get_price取数的正确性
stock_list = get_all_securities(date=by_date).index.tolist()

CPU times: user 990 ms, sys: 0 ns, total: 990 ms
Wall time: 1.13 s


In [16]:
#
new_high_list_dic={}
newhigh_percent = pd.Series()
#
prices = get_price(stock_list, end_date=end_date, frequency='daily', 
                   fields='close', count=window+check_days, panel=False
                  ).pivot(index='time', columns='code',values='close').dropna(axis=1)
#
prices.tail(3)

code,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,000012.XSHE,000014.XSHE,000016.XSHE,000017.XSHE,000019.XSHE,000020.XSHE,000021.XSHE,000023.XSHE,000025.XSHE,000026.XSHE,000027.XSHE,000028.XSHE,000029.XSHE,000030.XSHE,000031.XSHE,000032.XSHE,000034.XSHE,000035.XSHE,000036.XSHE,000037.XSHE,000038.XSHE,000039.XSHE,000040.XSHE,000042.XSHE,000045.XSHE,000046.XSHE,000048.XSHE,000049.XSHE,000050.XSHE,000055.XSHE,...,688607.XSHG,688608.XSHG,688609.XSHG,688611.XSHG,688616.XSHG,688617.XSHG,688618.XSHG,688619.XSHG,688626.XSHG,688628.XSHG,688630.XSHG,688633.XSHG,688636.XSHG,688656.XSHG,688658.XSHG,688659.XSHG,688661.XSHG,688662.XSHG,688663.XSHG,688665.XSHG,688667.XSHG,688668.XSHG,688669.XSHG,688676.XSHG,688677.XSHG,688678.XSHG,688679.XSHG,688680.XSHG,688683.XSHG,688686.XSHG,688687.XSHG,688689.XSHG,688696.XSHG,688698.XSHG,688699.XSHG,688777.XSHG,688788.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-05-19,14.62,19.25,9.90,1.87,4.46,7.31,2.31,11.90,3.45,12.82,6.09,9.74,4.79,3.76,8.47,8.43,10.62,11.10,12.16,8.92,5.97,28.02,9.28,5.30,3.89,13.58,14.68,4.45,4.12,7.26,4.45,13.04,2.97,9.82,6.84,1.53,18.44,35.73,9.12,4.25,...,18.30,138.62,8.18,56.21,10.25,187.73,39.83,13.72,32.15,24.50,47.87,41.20,109.90,37.51,19.54,9.13,54.92,27.21,26.85,123.38,120.95,57.05,22.89,17.35,76.94,18.66,10.36,227.99,20.50,174.05,17.30,28.12,269.30,16.89,95.86,69.58,58.70,30.37,42.44,41.56
2022-05-20,15.02,18.90,10.38,1.88,4.38,7.32,2.40,11.86,3.41,12.67,6.15,9.49,4.88,3.48,8.55,8.41,10.74,11.08,12.21,9.10,6.13,28.34,9.04,5.36,3.88,13.65,14.81,4.45,4.10,7.07,4.54,13.20,3.06,9.42,7.00,1.51,18.45,37.39,9.21,4.21,...,18.44,137.78,8.15,57.13,10.34,188.15,40.41,13.50,33.20,24.59,47.95,41.45,111.24,37.80,19.04,9.18,54.77,28.15,27.10,129.28,119.47,57.05,23.39,17.53,79.00,18.77,10.47,223.30,20.51,176.58,17.40,28.30,277.65,16.90,95.67,71.42,59.18,30.58,42.88,43.05
2022-05-23,14.83,18.35,10.13,1.95,4.36,7.33,2.43,11.93,3.34,12.58,6.17,9.31,5.22,3.40,8.84,8.57,10.80,10.93,12.56,9.09,6.08,28.50,8.93,5.48,3.84,13.97,14.99,4.44,4.08,6.92,4.61,13.22,3.07,9.96,6.97,1.55,18.63,37.73,9.20,4.24,...,19.17,137.40,8.27,57.77,10.30,192.01,40.46,13.69,33.47,25.07,49.25,42.50,112.30,38.41,19.21,9.22,56.32,30.29,26.42,130.33,125.62,57.03,23.99,17.34,81.40,19.02,10.49,214.69,20.63,173.30,18.12,28.83,281.02,16.99,95.70,71.61,59.54,31.74,43.10,42.04


In [17]:
for i in range(check_days):
    check_date = prices.index[window+i].date()
    # 取出统计窗口内的数据，即从 check_day 开始，向前推 window 个日期的切片
    price = prices.iloc[i+1:window+i+1]
    # 核心：
    # 最大值的索引==长度，即最新日创价格新高
    # 前一日往前的最大值索引<gap间隔之前的长度，即gap前期有新高值，当前值
    s_result = price.apply(lambda x: np.argmax(x.values) == (len(x) -1) and np.argmax(x.values[:-1])<(len(x)-1-gap))
    new_high_list = s_result[s_result].index.tolist()
    newhigh_percent.loc[check_date] = 100*(len(new_high_list) / len(stock_list))
    new_high_list_dic[check_date]=new_high_list

In [18]:
# 创新高的比例
newhigh_percent#.tail()

2022-04-28    0.070505
2022-04-29    0.094007
2022-05-05    0.141011
2022-05-06    0.047004
2022-05-09    0.094007
2022-05-10    0.117509
2022-05-11    0.047004
2022-05-12    0.117509
2022-05-13    0.000000
2022-05-16    0.141011
2022-05-17    0.047004
2022-05-18    0.141011
2022-05-19    0.070505
2022-05-20    0.117509
2022-05-23    0.117509
dtype: float64

In [20]:
# 创新高的股票列表
new_high_list_dic#[datetime.datetime.strptime(end_date,"%Y-%m-%d").date()]

{datetime.date(2022, 4, 28): ['000756.XSHE', '000839.XSHE', '603086.XSHG'],
 datetime.date(2022, 4, 29): ['002693.XSHE',
  '002737.XSHE',
  '300284.XSHE',
  '603706.XSHG'],
 datetime.date(2022, 5, 5): ['002365.XSHE',
  '002789.XSHE',
  '600018.XSHG',
  '600233.XSHG',
  '600486.XSHG',
  '600780.XSHG'],
 datetime.date(2022, 5, 6): ['002562.XSHE', '603299.XSHG'],
 datetime.date(2022, 5, 9): ['000985.XSHE',
  '002216.XSHE',
  '601579.XSHG',
  '601975.XSHG'],
 datetime.date(2022, 5, 10): ['600371.XSHG',
  '600422.XSHG',
  '600770.XSHG',
  '600820.XSHG',
  '603688.XSHG'],
 datetime.date(2022, 5, 11): ['300239.XSHE', '300778.XSHE'],
 datetime.date(2022, 5, 12): ['000553.XSHE',
  '000680.XSHE',
  '002883.XSHE',
  '600892.XSHG',
  '603398.XSHG'],
 datetime.date(2022, 5, 13): [],
 datetime.date(2022, 5, 16): ['000552.XSHE',
  '002496.XSHE',
  '600389.XSHG',
  '600628.XSHG',
  '600720.XSHG',
  '601872.XSHG'],
 datetime.date(2022, 5, 17): ['002183.XSHE', '603556.XSHG'],
 datetime.date(2022, 5, 18)